In [1]:
from google.colab import drive
drive.mount("/content/d")

Mounted at /content/d


In [70]:
from IPython.display import clear_output
!pip install jupyter-dash
!pip install category-encoders
clear_output()

In [78]:
import json
import random
import requests
import time
import os

if False:
    import plotly.express as px
    from jupyter_dash import JupyterDash
    from dash import dcc
    from dash import html
    from dash.dependencies import Input, Output

from collections import defaultdict
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

RANDOM_STATE = 13

# DATA COLLECTION

In [ ]:
start_time = time.time()
print(f"PROCESS STARTED...")

def getPage(page = 0):
    params = {
        'text': 'NAME:Программист',
        'area': 1511,
        'page': page,
        'per_page': 100 
    }
    
    req = requests.get('https://api.hh.ru/vacancies', params) 
    data = req.content.decode() 
    req.close()
    return data

for page in range(0, 10):
    
    jsObj = json.loads(getPage(page))
    nextFileName = 'd/MyDrive/hhApi/docs/pagination/{}.json'.format(len(os.listdir('d/MyDrive/hhApi/docs/pagination')))
    
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    
    if (jsObj['pages'] - page) <= 1:
        break
    time.sleep(random.uniform(0.5, 2))


for fl in os.listdir('d/MyDrive/hhApi/docs/pagination'):
    
    f = open('d/MyDrive/hhApi/docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    
    jsonObj = json.loads(jsonText)
    for v in jsonObj['items']:
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
        
        fileName = 'd/MyDrive/hhApi/docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()

        time.sleep(random.uniform(0.5, 2))
        
print(f"...PROCESS COMPLETED, time: {int(time.time() - start_time)}s")

PROCESS STARTED...
...PROCESS COMPLETED, time: 205s


In [ ]:
fPath = "d/MyDrive/hhApi/docs/vacancies"

for filename in os.listdir("d/MyDrive/hhApi/docs/vacancies"):
    os.rename(os.path.join(fPath,filename), os.path.join(fPath,"1948_"+filename))

In [ ]:
!mv d/MyDrive/hhApi/docs/vacancies/* d/MyDrive/hhApi/docs/stack/

In [ ]:
!ls d/MyDrive/hhApi/docs/pagination

In [ ]:
!rm d/MyDrive/hhApi/docs/pagination/*

rm: cannot remove 'd/MyDrive/hhApi/docs/pagination/*': No such file or directory


In [ ]:
print(len(os.listdir("d/MyDrive/hhApi/docs/stack")))
print(len(os.listdir("d/MyDrive/hhApi/docs/vacancies")))

13852
0


In [ ]:
fPath = "d/MyDrive/hhApi/docs/vacancies/"
cnt = 0

for f in os.listdir(fPath):
    j = json.load(open(fPath + f, "rb"))
    if "errors" in j.keys():
        print(cnt)
        cnt+=1 
        # os.remove(fPath+f)

print(cnt)

0


In [122]:
dd.columns

Index(['name', 'city', 'salary_from', 'salary_to', 'salary_currency',
       'salary_is_gross', 'experience', 'schedule', 'employment_type',
       'employer_name',
       ...
       'skill_эдо', 'skill_электроника', 'skill_электронный документооборот',
       'skill_электропривод', 'skill_электротехника',
       'skill_электрощитовое оборудование',
       'skill_эффективное решение проблем', 'skill_юмор', 'skill_язык с',
       'skill_яндекс.метрика'],
      dtype='object', length=1489)

# DATA PROCESSING

In [39]:
''' EXTRACT DATA FROM JSON-s TO THE DF '''
res = pd.DataFrame(
    dtype=object,
    columns = [
        "name", "city", "salary_from", "salary_to", "salary_currency", "salary_is_gross",
        "location_address", "experience", "schedule", "employment_type",
        "skills_required", "specializations", "professional_roles", "employer_name",
        "employer_is_trusted", "working_days", "working_time_intervals",
        "working_time_modes", "languages"
    ]    
)

start_time = time.time()
print(f"PROCESS STARTED...")

fPath = "d/MyDrive/hhApi/docs/stack/"

for f in os.listdir(fPath):
    # print(f)
    try:
        j = json.load(open(fPath + f, "rb"))
        name = j["name"] if "name" in j.keys() else "-1"
        city = j["area"]["name"] if "area" in j.keys() else "-1"
        salary_from = j["salary"]["from"] if j["salary"] != None else "-1"  #all(["salary" in j.keys(), "from" in j["salary"].keys()]) else "-1"
        salary_to = j["salary"]["to"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "to" in j["salary"].keys()]) else "-1"
        salary_currency = j["salary"]["currency"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "currency" in j["salary"].keys()]) else "-1"
        salary_is_gross = j["salary"]["gross"] if j["salary"] != None else "-1" #all(["salary" in j.keys(), "gross" in j["salary"].keys()]) else "-1"
        location_address = j["address"] if "address" in j.keys() else "-1"
        experience = j["experience"]["id"] if "experience" in j.keys() else "-1"
        schedule = j["schedule"]["id"] if "schedule" in j.keys() else "-1"
        employment_type = j["employment"]["id"] if "employment" in j.keys() else "-1"
        skills_required = [skill["name"] for skill in j['key_skills']] if "key_skills" in j.keys() else "-1"
        specializations = [sp["name"] for sp in j["specializations"]] if "specializations" in j.keys() else "-1"
        professional_roles = [rl["name"] for rl in j['professional_roles']] if "professional_roles" in j.keys() else "-1"
        employer_name = j["employer"]["name"] if "employer" in j.keys() else "-1"
        employer_is_trusted = j["employer"]["trusted"] if all(["employer" in j.keys(), "trusted" in j["employer"].keys()]) else "-1"
        working_days = j["working_days"]
        working_time_intervals = j["working_time_intervals"]
        working_time_modes = j["working_time_modes"]
        languages = j["languages"]

        res.loc[len(res)]  = (
            name, city, salary_from, salary_to, salary_currency, salary_is_gross, 
            location_address, experience, schedule, employment_type, skills_required,
            specializations, professional_roles, employer_name, employer_is_trusted,
            working_days, working_time_intervals, working_time_modes, languages)
        
    except:
        # continue
        print("ATENCION")
        break

print(f"...PROCESS COMPLETED, time: {int(time.time() - start_time)}s")
res

PROCESS STARTED...
...PROCESS COMPLETED, time: 184s


,name,city,salary_from,salary_to,salary_currency,salary_is_gross,location_address,experience,schedule,employment_type,skills_required,specializations,professional_roles,employer_name,employer_is_trusted,working_days,working_time_intervals,working_time_modes,languages
0,Программист 1С,Москва,-1,-1,-1,-1,None,between3And6,fullDay,full,[],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Агранта,True,[],[],[],[]
1,Web разработчик на C++,Москва,-1,-1,-1,-1,None,between3And6,remote,full,[C++],"[Программирование, Разработка, Системы управле...","[Программист, разработчик]",РостДжоб,True,[],[],[],"[{'id': 'eng', 'name': 'Английский', 'level': ..."
2,Senior Frontend (Angular) developer,Москва,-1,-1,-1,-1,"{'city': 'Москва', 'street': 'Филёвская линия'...",between3And6,fullDay,full,"[Scrum, HTML5, AngularJS, GraphQL, JavaScript,...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",СБЕР (ООО еАптека),True,[],[],[],[]
3,Администратор СЭД Directum RX / Разработчик со...,Москва,-1,-1,-1,-1,"{'city': 'Москва', 'street': 'Причальный проез...",between1And3,fullDay,full,"[MS SQL Server, MS SQL, Опытный пользователь П...","[Розничная торговля, Прямые продажи, Менеджер ...",[Другое],WINLINE,True,[],[],[],[]
4,Программист 1С (ERP),Москва,-1,-1,-1,-1,None,between3And6,fullDay,full,"[1С: Управление Производственным Предприятием,...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",Завод ПРОТЕЙ,True,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13847,Middle frontend developer,Санкт-Петербург,110000,150000,RUR,False,None,between3And6,remote,full,"[JavaScript, React JS, CSS, Next, Redux, Web 3...","[Розничная торговля, Прямые продажи, Менеджер ...",[Другое],CoreBlocks,False,[],"[{'id': 'from_four_to_six_hours_in_a_day', 'na...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': 'eng', 'name': 'Английский', 'level': ..."
13848,Программист PHP,Санкт-Петербург,-1,-1,-1,-1,"{'city': 'Кудрово', 'street': 'проспект Строит...",noExperience,fullDay,full,"[JavaScript, jQuery, PHP, MySQL, HTML, MSSQL, ...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",АВТОПОЛЕ,True,[],[],[],"[{'id': 'eng', 'name': 'Английский', 'level': ..."
13849,RND разработчик (C/C++),Санкт-Петербург,-1,-1,-1,-1,None,between3And6,fullDay,full,"[PostgreSQL, Linux, СУБД, Ansi C++]","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",SberTech,True,[],[],[],[]
13850,Разработчик БД MySQL (Senior),Санкт-Петербург,170000,250000,RUR,False,None,between3And6,fullDay,full,"[MySQL, SQL, Работа с базами данных, СУБД, EXP...","[Программирование, Разработка, Системы управле...","[Программист, разработчик]",4people,True,[],[],[],[]


In [40]:
# drop empty targets
df = res[(res.salary_from != "-1") & (res.salary_to != "-1")]
print(df.shape)
df = df[df.salary_from.notna()].reset_index(drop=True)
print(df.shape)

(5891, 19)
(5055, 19)


In [41]:
# drop useless features
df = df.reset_index()
df = df.drop(["location_address", "working_days", "working_time_intervals", "working_time_modes", "languages"], axis=1)
print(df.shape)

(5055, 15)


In [42]:
# explode and one-hot 'specializations' column
d = df.explode("specializations")
d = d[["index", "specializations"]]
d = d[['index']].join(pd.get_dummies(d['specializations']).add_prefix('specialization_')).groupby('index').max()
df = pd.concat([df, d], axis=1)
print(df.shape)

(5055, 90)


In [43]:
# explode and one-hot 'professional_roles' column
d = df.explode("professional_roles")
d = d[["index", "professional_roles"]]
d = d[['index']].join(pd.get_dummies(d['professional_roles']).add_prefix('role_')).groupby('index').max()
df = pd.concat([df, d], axis=1)
print(df.shape)

(5055, 127)


In [44]:
# explode and one-hot 'skills_required' column

d = df.explode("skills_required")
d.skills_required = d.skills_required.apply(lambda x: str(x).lower())
skills = d.skills_required.unique()
skills = skills.tolist()
skills = [x for x in skills if x != "nan"]

skills_dct = dict(zip(skills, skills))

for s in [x for x in skills if any(["1c" in x, "1с" in x])]:
    skills_dct[s] = "1c"
for s in [x for x in skills if any(["sql" in x])]:
    skills_dct[s] = "sql"
for s in [x for x in skills if all(["java" in x, "javascript" not in x])]:
    skills_dct[s] = "java"
for s in [x for x in skills if all(["javascript" in x])]:
    skills_dct[s] = "javascript"
for s in [x for x in skills if all(["linux" in x])]:
    skills_dct[s] = "linux"
for s in [x for x in skills if all(["php" in x])]:
    skills_dct[s] = "php"
for s in [x for x in skills if any(["английск" in x, "engl" in x])]:
    skills_dct[s] = "english"
for s in [x for x in skills if any(["css" in x])]:
    skills_dct[s] = "css"
for s in [x for x in skills if any(["c++" in x])]:
    skills_dct[s] = "c++"
for s in [x for x in skills if any(["docker" in x])]:
    skills_dct[s] = "docker"
for s in [x for x in skills if any(["html" in x])]:
    skills_dct[s] = "html"
for s in [x for x in skills if any(["xml" in x])]:
    skills_dct[s] = "xml"
for s in [x for x in skills if any(["node" in x])]:
    skills_dct[s] = "nodejs"
for s in [x for x in skills if any(["react" in x])]:
    skills_dct[s] = "react"
for s in [x for x in skills if any([".net" in x])]:
    skills_dct[s] = ".net"
for s in [x for x in skills if any(["rest" in x])]:
    skills_dct[s] = "rest"
for s in [x for x in skills if any(["баз" in x])]:
    skills_dct[s] = "работа с базами данных"
for s in [x for x in skills if any(["android" in x])]:
    skills_dct[s] = "android"
for s in [x for x in skills if any(["многозада" in x])]:
    skills_dct[s] = "умение работать в условиях многозадачности"
for s in [x for x in skills if any(["vue" in x])]:
    skills_dct[s] = "vue.js"
for s in [x for x in skills if any(["kafka" in x])]:
    skills_dct[s] = "kafka"
for s in [x for x in skills if any(["front" in x])]:
    skills_dct[s] = "frontend"
for s in [x for x in skills if any(["golang" in x])]:
    skills_dct[s] = "go"
for s in [x for x in skills if any(["django" in x])]:
    skills_dct[s] = "django"
for s in [x for x in skills if any(["документац" in x])]:
    skills_dct[s] = "разработка документации"
for s in [x for x in skills if any(["jira" in x])]:
    skills_dct[s] = "jira"
for s in [x for x in skills if any(["delphi" in x])]:
    skills_dct[s] = "delphi"
for s in [x for x in skills if any(["oracle" in x])]:
    skills_dct[s] = "oracle"
for s in [x for x in skills if any(["unity" in x])]:
    skills_dct[s] = "unity"
for s in [x for x in skills if any(["3d" in x])]:
    skills_dct[s] = "3d"
# for s in list(cnt[cnt < 20].keys()):
#     skills_dct[s] = "прочее"

d.skills_required = d.skills_required.map(skills_dct)

d = d[["index", "skills_required"]]
d = d[['index']].join(pd.get_dummies(d['skills_required']).add_prefix('skill_')).groupby('index').max()
df = pd.concat([df, d], axis=1)

df = df.drop(["specializations", "professional_roles", "skills_required"], axis=1)

print(df.shape)

(5055, 1490)


In [45]:
def count_values(df, what:int):
    ''' value counts for the one-hot columns '''
    wt = {1: "specialization", 2: "role", 3: "skill"}
    for k,v in df[[x for x in df if wt[what] in x]].sum().sort_values(ascending=False).to_dict().items():
        print(v, "---", k) 

    return None

def get_columns(df, what:int, how_many:int):
    ''' get the first 'how_many' by frequency values '''
    wt = {1: "specialization", 2: "role", 3: "skill"}
    clms = df[[x for x in df if wt[what] in x]].sum().sort_values(ascending=False).iloc[how_many:].index
    df = df.drop(clms, axis=1)
    
    return df

count_values(df, 3)

1586 --- skill_sql
1289 --- skill_1c
1128 --- skill_git
737 --- skill_javascript
651 --- skill_php
648 --- skill_linux
559 --- skill_html
534 --- skill_c++
521 --- skill_ооп
455 --- skill_english
422 --- skill_css
381 --- skill_python
376 --- skill_работа с базами данных
283 --- skill_c#
227 --- skill_react
224 --- skill_typescript
209 --- skill_java
208 --- skill_.net
205 --- skill_docker
195 --- skill_rest
189 --- skill_jquery
169 --- skill_nodejs
168 --- skill_xml
164 --- skill_qt
161 --- skill_laravel
151 --- skill_vue.js
149 --- skill_умение работать в условиях многозадачности
143 --- skill_redis
140 --- skill_работа в команде
121 --- skill_mongodb
116 --- skill_rabbitmq
108 --- skill_разработка документации
100 --- skill_api
98 --- skill_symfony
97 --- skill_android
94 --- skill_разработка по
92 --- skill_go
88 --- skill_удаленная работа
82 --- skill_kafka
80 --- skill_3d
79 --- skill_субд
78 --- skill_js
78 --- skill_kotlin
78 --- skill_backend
76 --- skill_скд
71 --- skill_matl

# REGRESSION MODEL

In [125]:
''' OPTIMAL MODEL FEATURE ENGINE DETECTION '''

def evaluate_model(model, x, y, x_vl=None, y_vl=None):
    ''' trivial pipe for model evaluation '''
    if x_vl is None:
        x_tr, x_vl, y_tr, y_vl = train_test_split(x, y, test_size=0.2, random_state=RANDOM_STATE)
    else:
        x_tr, y_tr = x, y
    pipe = Pipeline([("scaler", StandardScaler()), ("model", model())])
    pipe.fit(x_tr, y_tr)
    preds = pipe.predict(x_vl)

    return int(mean_squared_error(y_vl, preds))

models = [LinearRegression, RandomForestRegressor, 
          MLPRegressor, LGBMRegressor, XGBRegressor]

metrics = {}
for model in models:
    print(f"{model.__name__}: {evaluate_model(model, x, y):,}")
    metrics[model.__name__] = evaluate_model(model, x, y)

print(f"\nbest score {min(metrics, key=metrics.get)}: {metrics[min(metrics, key=metrics.get)]}")

LinearRegression: 159,120,657
RandomForestRegressor: 164,099
MLPRegressor: 9,658,536,378
LGBMRegressor: 1,870,202
[19:02:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor: 24,059
[19:03:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

best score XGBRegressor: 24059


-> REGRESSOR BASED ON XGBOOST WILL BE APPLIED